In [ ]:
import mysql.connector

connection = mysql.connector.connect(
            host="crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com",
            user="Saam",
            passwd="Stitcheson12!",
            database="Crypto"
        )

cursor = connection.cursor()

create_financing_table = """
CREATE TABLE IF NOT EXISTS crypto_lending_borrowing (
    id INT AUTO_INCREMENT PRIMARY KEY,  
    crypto_symbol VARCHAR(10),                     -- Symbol of the cryptocurrency
    date DATETIME NOT NULL,                        -- Date and time of the data entry
    lending_rate DECIMAL(10, 6) NOT NULL,          -- Lending rate (interest rate for lenders)
    borrowing_rate DECIMAL(10, 6) NOT NULL,        -- Borrowing rate (interest rate for borrowers)
    utilization_rate DECIMAL(10, 6),               -- Percentage of liquidity that has been borrowed
    stable_borrow_rate DECIMAL(10, 6),        

    UNIQUE KEY (crypto_symbol, date),           -- Composite primary key
    FOREIGN KEY (crypto_symbol) REFERENCES crypto_reference(crypto_symbol) -- Foreign key constraint
);"""

cursor.execute(create_financing_table)

connection.commit()
cursor.close()
connection.close()

In [1]:
import os
import pandas as pd
import numpy as np

file_path = os.getcwd()

rates_path = f"{file_path}\\rates_data_v4.csv"

rates_df = pd.read_csv(rates_path)

In [2]:
print(rates_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1048575 non-null  int64  
 1   Symbol                  1048575 non-null  object 
 2   Timestamp               1048575 non-null  object 
 3   liquidityRate_avg       1048575 non-null  float64
 4   variableBorrowRate_avg  1048575 non-null  float64
 5   utilizationRate_avg     1048575 non-null  float64
 6   stableBorrowRate_avg    1048575 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 56.0+ MB
None


In [3]:
rates_df["date"] = pd.to_datetime(rates_df["Timestamp"])

In [4]:
rates_df.drop(columns=["Timestamp", "Unnamed: 0"], inplace=True)

In [5]:
rates_df.head()

,Symbol,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg,date
0,DAI,0.000899,0.003996,0.083333,0.042083,2020-12-01 16:00:00
1,DAI,0.000000,0.000000,0.250000,0.046250,2020-12-01 17:00:00
2,DAI,0.000000,0.000000,0.250000,0.046250,2020-12-01 18:00:00
3,DAI,0.000000,0.000000,0.250000,0.046250,2020-12-01 19:00:00
4,DAI,0.000000,0.000000,0.250000,0.046250,2020-12-01 20:00:00


In [6]:
min_date = rates_df["date"].min()
max_date = rates_df["date"].max()

print(min_date, max_date)

date_range = pd.date_range(start=min_date, end=max_date, freq='H')

print(len(date_range))

date_unique = rates_df["date"].nunique()

print(date_unique)

2020-11-30 23:00:00 2024-10-31 13:00:00
34335
34335


In [7]:
nan_counts = rates_df.isna().sum()
print("NaN counts in each column:")
print(nan_counts)

NaN counts in each column:
Symbol                    0
liquidityRate_avg         0
variableBorrowRate_avg    0
utilizationRate_avg       0
stableBorrowRate_avg      0
date                      0
dtype: int64


In [8]:
zero_counts = (rates_df == 0).sum()
print("Zero counts in each column:")
print(zero_counts)

Zero counts in each column:
Symbol                         0
liquidityRate_avg         176183
variableBorrowRate_avg    180303
utilizationRate_avg          249
stableBorrowRate_avg      177049
date                           0
dtype: int64


In [10]:
def max_consecutive_zeros(df):
    max_zeros = {}
    
    for col in df.columns:
        # Create a mask of where values are zero
        mask = df[col] == 0
        
        # Use the mask to find groups of consecutive zeros
        groups = mask.astype(int).groupby((mask != mask.shift()).cumsum()).sum()
        
        # Get the maximum length of consecutive zeros
        max_zeros[col] = groups.max() if not groups.empty else 0
        
    return max_zeros

# Example usage
max_zeros_counts = max_consecutive_zeros(rates_df)
print("Maximum consecutive zeros in each column:")
print(max_zeros_counts)

Maximum consecutive zeros in each column:
{'Symbol': 0, 'liquidityRate_avg': 1548, 'variableBorrowRate_avg': 34340, 'utilizationRate_avg': 57, 'stableBorrowRate_avg': 34321, 'date': 0}


In [20]:
import pandas as pd

# Sample DataFrame
# rates_df = pd.DataFrame({
#     'symbol': ['A', 'B', 'A', 'B', 'A', 'C', 'C', 'C', 'A', 'A'],
#     'lending_rate': [0, 0.05, 0, 0, 0.02, 0, 0, 0, 0, 0],
#     'borrowing_rate': [0, 0, 0, 0.03, 0, 0, 0, 0, 0, 0.01]
# })

def count_consecutive_zeros(series):
    """Count the maximum number of consecutive zeros in a Series."""
    max_count = 0
    current_count = 0

    for value in series:
        if value == 0:
            current_count += 1
            max_count = max(max_count, current_count)
        else:
            current_count = 0

    return max_count

# Initialize an empty list to store results
results = []

# Group by 'symbol'
grouped = rates_df.groupby('Symbol')

# Iterate over each numeric column
for column in rates_df.select_dtypes(include='number').columns:
    # Calculate zeros and max consecutive zeros for each symbol and column
    summary = grouped[column].agg(
        zero_count=lambda x: (x == 0).sum(),  # Count of zeros
        max_consecutive_zeros=count_consecutive_zeros,  # Maximum consecutive zeros
        count_ = lambda x: (x.notnull()).sum()
    ).reset_index()
    
    # Add column name to the results
    summary['column'] = column
    results.append(summary)

# Concatenate all results into a single DataFrame
final_result = pd.concat(results, ignore_index=True)

# Reorder columns for clarity
final_result = final_result[['Symbol', 'column', 'zero_count', 'max_consecutive_zeros', 'count_']]

final_result_sorted = final_result.sort_values(by=['max_consecutive_zeros', 'zero_count'], ascending=False)

from IPython.display import HTML

# Display DataFrame as scrollable output
display(HTML(final_result_sorted.to_html(max_rows=180, max_cols=5)))


,Symbol,column,zero_count,max_consecutive_zeros,count_
31,AAAVE,variableBorrowRate_avg,34319,34319,34319
32,AAVE,variableBorrowRate_avg,34319,34319,34319
93,AAAVE,stableBorrowRate_avg,34319,34319,34319
94,AAVE,stableBorrowRate_avg,34319,34319,34319
116,SUSD,stableBorrowRate_avg,34257,34257,34257
115,SNX,stableBorrowRate_avg,34203,34203,34203
96,ABUSD,stableBorrowRate_avg,19967,19967,34056
107,BUSD,stableBorrowRate_avg,19967,19967,34056
0,AAAVE,liquidityRate_avg,33063,1548,34319
1,AAVE,liquidityRate_avg,33063,1548,34319


In [57]:
import pymc
import arviz as az
import matplotlib as mpl

Variable Borrow Rate={ 
Base Rate+ U*Multiplier + (if U > threshold for U High multiplier * U)
​
if Utilization<Kink
if Utilization≥Kink}
​
liquidation rate = variable borrow rate 
 
variable_borrow_rate = alpha + ß * X  + (Max(U-ß1, 0)/(U-ß1))* ß2 * U

liqudidation_rate = alpha + ß * variable_borrow_rate

variable_borrow_rate = alpha + ß * liqudidation_rate

1.  find missing values
2. create lists of consecutive missing entries
3. look left and right from missing entries, try to get length in each direction equivalent to length of missing entries or nearest amount and concatenate these together
4. drop sparse entries
5. for missing variable borrow rate: calculate parameters for variable borrow rate from U, 
calculate parameters for variable borrow rate from liduidation rate
for missing variable liquidation rate: calculate paramaters for liqudidation rate from U, calculate paramters for variable borrow rate
6. check for Error, accept imputation according to threshold
7. predict in for missing entries if either rate exists first (less entropy) and if not exists use U for prediction, if neither exists, mask entries using value like nan or any real number not present in the dataset i.e. -1

In [ ]:
class MissingDataImputer:
    def __init__(self, df, symbol_column='Symbol'):
        self.df = df
        self.symbol_column = symbol_column
        self.missing_indices = {'variable_borrow_rate': [], 'liquidation_rate': []}
        self.training_data = {'variable_borrow_rate': [], 'liquidation_rate': []}
        self._identify_missing_windows()  # Initialize by finding missing windows

    def _identify_missing_windows(self):
        # For each column we want to check, find missing windows for each symbol
        symbols = self.df[self.symbol_column].unique()
        
        for sym in symbols:
            sym_df = self.df[self.df[self.symbol_column] == sym].reset_index(drop=True)
            for column in self.missing_indices.keys():
                missing_indices = self._get_consecutive_missing(sym_df, column)
                # Store the missing indices for each symbol under a common structure
                self.missing_indices[column].extend(missing_indices)
                # Gather training data around the missing windows
                self._collect_surrounding_training_data(sym_df, missing_indices, column)

    def _get_consecutive_missing(self, sym_df, column):
        # Find start and end indices of consecutive missing values in the column
        missing_indices = []
        start = None
        is_missing = sym_df[column].isna()
        
        for i, missing in enumerate(is_missing):
            if missing and start is None:
                start = i
            elif not missing and start is not None:
                missing_indices.append((start, i - 1))
                start = None
        if start is not None:
            missing_indices.append((start, len(is_missing) - 1))
        
        return missing_indices

    def _collect_surrounding_training_data(self, sym_df, missing_indices, column):
        # Collect surrounding data as training data for each missing window
        for start, end in missing_indices:
            window_length = end - start + 1
            
            # Left and right data retrieval
            left_data = sym_df[column].iloc[max(0, start - window_length):start]
            right_data = sym_df[column].iloc[end + 1:min(len(sym_df), end + 1 + window_length)]
            
            # Combine left and right data
            surrounding_data = pd.concat([left_data, right_data]).dropna().values
            self.training_data[column].append(surrounding_data)

    def get_missing_info(self):
        # Return a dictionary of missing indices and surrounding training data
        return {
            'missing_indices': self.missing_indices,
            'training_data': self.training_data
        }




